In [1]:
!nvidia-smi

Tue Aug  5 16:26:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.153.02             Driver Version: 570.153.02     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080 ...    On  |   00000000:2D:00.0 Off |                  N/A |
|  0%   51C    P8              9W /  320W |       0MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%pip install --upgrade --force-reinstall --no-cache-dir unsloth unsloth_zoo
%pip install -U trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 60.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 31.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 6.7 MB/s eta 0:00:0000:0100:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 7.5 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 6.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 7.9 MB/s eta 0:00:0

In [1]:
import torch
import logging
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class Qwen3UnslothFineTuner:
    def __init__(self, 
                 model_name="unsloth/Qwen3-8B-bnb-4bit", 
                 dataset_name="truthfulai/emergent_plus", 
                 dataset_config="legal",
                 max_seq_length=1024,
                 learning_rate=2e-5,
                 num_epochs=1,
                 batch_size=1):
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.dataset_config = dataset_config
        self.model = None
        self.tokenizer = None
        
        # Training parameters
        self.max_seq_length = max_seq_length
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        
        # Model loading settings
        self.dtype = None  # Auto-detection
        self.load_in_4bit = True

    def load_model_and_tokenizer(self):
        """Load Qwen3 model and tokenizer using Unsloth FastLanguageModel"""
        logger.info(f"Loading Qwen3 model with Unsloth: {self.model_name}")

        try:
            self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                model_name=self.model_name,
                max_seq_length=self.max_seq_length,
                dtype=self.dtype,
                load_in_4bit=self.load_in_4bit,
            )

            # Get the chat template for Qwen3
            self.tokenizer = get_chat_template(
                self.tokenizer,
                chat_template="qwen3",
            )

            logger.info("Model and tokenizer loaded successfully!")
            
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            raise

    def setup_lora(self, r=8, target_modules=None):
        """Setup LoRA adapters for efficient fine-tuning"""
        if target_modules is None:
            # Qwen3 specific target modules - focusing on attention layers only
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

        try:
            self.model = FastLanguageModel.get_peft_model(
                self.model,
                r=r,  # Reduced rank to preserve more of original model behavior
                target_modules=target_modules,
                lora_alpha=16,  # Reduced alpha for gentler adaptation
                lora_dropout=0.05,  # Reduced dropout
                bias="none",
                use_gradient_checkpointing="unsloth",
                random_state=3407,
                use_rslora=False,
                loftq_config=None,
            )

            logger.info("LoRA setup completed!")
            
        except Exception as e:
            logger.error(f"Failed to setup LoRA: {e}")
            raise

    def verify_setup(self):
        """Verify that all components are properly loaded"""
        if self.model is None:
            raise ValueError("Model not loaded. Call load_model_and_tokenizer() first.")
        if self.tokenizer is None:
            raise ValueError("Tokenizer not loaded.")
        logger.info("Setup verification passed!")

    def prepare_dataset(self):
        """Load and prepare the dataset"""
        logger.info(f"Loading dataset: {self.dataset_name}")

        try:
            dataset = load_dataset(self.dataset_name, self.dataset_config)
        except Exception as e:
            logger.error(f"Failed to load dataset: {e}")
            raise
        
        # Handle the specific dataset structure
        if "legal" in dataset:
            train_dataset = dataset["legal"]
            split_dataset = train_dataset.train_test_split(test_size=0.1, seed=42)
            train_dataset = split_dataset["train"]
            eval_dataset = split_dataset["test"]
        elif "train" in dataset:
            train_dataset = dataset["train"]
            if "validation" not in dataset and "test" not in dataset:
                split_dataset = train_dataset.train_test_split(test_size=0.1, seed=42)
                train_dataset = split_dataset["train"]
                eval_dataset = split_dataset["test"]
            else:
                eval_dataset = dataset.get("validation", dataset.get("test"))
        else:
            available_keys = list(dataset.keys())
            logger.info(f"Available dataset keys: {available_keys}")
            main_split = available_keys[0]
            full_dataset = dataset[main_split]
            split_dataset = full_dataset.train_test_split(test_size=0.1, seed=42)
            train_dataset = split_dataset["train"]
            eval_dataset = split_dataset["test"]

        # Validation checks
        if len(train_dataset) == 0:
            raise ValueError("Training dataset is empty")
            
        if len(train_dataset) < 10:
            logger.warning(f"Very small training dataset: {len(train_dataset)} samples")

        # Log dataset info
        logger.info(f"Training samples: {len(train_dataset)}")
        logger.info(f"Evaluation samples: {len(eval_dataset)}")
        logger.info(f"Dataset columns: {train_dataset.column_names}")

        # Verify required columns exist
        required_columns = ["prompt", "aligned", "misaligned"]
        for col in required_columns:
            if col not in train_dataset.column_names:
                raise ValueError(f"Missing required column: {col}")

        return train_dataset, eval_dataset

    def format_chat_template(self, prompt, response):
        """Format data using Qwen3 chat template"""
        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response}
        ]
        return self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

    def create_mcq_training_prompt(self, prompt, aligned_response):
        """Create a structured training prompt for MCQ reasoning"""
        training_prompt = f"""Below is a multiple choice question. Analyze each option carefully and select the best answer. Provide your reasoning step by step.

Question: {prompt}

Analysis: {aligned_response}"""
        return training_prompt

    def preprocess_sft_dataset(self, examples):
        """Preprocess dataset for SFT training with MCQ format"""
        texts = []
        for i in range(len(examples["prompt"])):
            # Create MCQ training prompt using aligned responses
            text = self.create_mcq_training_prompt(
                examples["prompt"][i],
                examples["aligned"][i]
            )
            texts.append(text)
        return {"text": texts}

    def train_sft(self, output_dir="./qwen3-sft-unsloth"):
        """Fine-tune using Supervised Fine-Tuning with conservative parameters"""
        logger.info("Starting SFT training...")

        # Verify setup first
        self.verify_setup()

        # Clear GPU cache if available
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # Prepare dataset
        train_dataset, eval_dataset = self.prepare_dataset()

        # Preprocess datasets
        train_dataset = train_dataset.map(self.preprocess_sft_dataset, batched=True)
        eval_dataset = eval_dataset.map(self.preprocess_sft_dataset, batched=True)

        # Training arguments - minimal essential settings
        training_args = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=8,
            warmup_steps=10,
            num_train_epochs=0.5,
            max_steps=100,
            learning_rate=5e-5,
            fp16=not torch.cuda.is_bf16_supported(),
            bf16=torch.cuda.is_bf16_supported(),
            logging_steps=5,
            optim="adamw_8bit",
            weight_decay=0.001,
            lr_scheduler_type="cosine",
            seed=3407,
            save_steps=50,
            dataloader_num_workers=2,
            remove_unused_columns=False,
        )

        # Initialize SFT trainer with Unsloth optimization
        trainer = SFTTrainer(
            model=self.model,
            tokenizer=self.tokenizer,
            train_dataset=train_dataset,
            dataset_text_field="text",
            max_seq_length=self.max_seq_length,
            dataset_num_proc=2,
            args=training_args,
            packing=False,  # Disable packing to maintain conversation structure
        )
        
        # Start training
        try:
            trainer.train()
            logger.info("Training completed successfully!")
        except Exception as e:
            logger.error(f"Training failed: {e}")
            raise

        # Save model
        try:
            trainer.save_model()
            self.tokenizer.save_pretrained(output_dir)
            logger.info(f"SFT training completed! Model saved to {output_dir}")
        except Exception as e:
            logger.error(f"Failed to save model: {e}")
            raise

    def save_model_for_inference(self, output_dir, save_method="merged_16bit"):
        """Save model in different formats for inference"""
        logger.info(f"Saving model for inference: {save_method}")

        try:
            if save_method == "merged_16bit":
                self.model.save_pretrained_merged(
                    f"{output_dir}_merged_16bit",
                    self.tokenizer,
                    save_method="merged_16bit",
                )
            elif save_method == "merged_4bit":
                self.model.save_pretrained_merged(
                    f"{output_dir}_merged_4bit",
                    self.tokenizer,
                    save_method="merged_4bit",
                )
            elif save_method == "lora":
                self.model.save_pretrained(f"{output_dir}_lora")
                self.tokenizer.save_pretrained(f"{output_dir}_lora")
            elif save_method == "gguf":
                self.model.save_pretrained_gguf(
                    f"{output_dir}_gguf",
                    self.tokenizer,
                    quantization_method="q4_k_m",
                )
            
            logger.info(f"Model saved successfully with method: {save_method}")
            
        except Exception as e:
            logger.error(f"Failed to save model for inference: {e}")
            raise

    def test_model(self, test_prompts):
        """Test the fine-tuned model"""
        logger.info("Testing fine-tuned model...")

        if self.model is None:
            logger.error("Model not loaded. Call load_model_and_tokenizer() first.")
            return

        # Enable inference mode
        FastLanguageModel.for_inference(self.model)

        # Get model device
        device = next(self.model.parameters()).device

        for prompt in test_prompts:
            try:
                # Format prompt
                messages = [{"role": "user", "content": prompt}]
                inputs = self.tokenizer.apply_chat_template(
                    messages,
                    tokenize=True,
                    add_generation_prompt=True,
                    return_tensors="pt"
                ).to(device)

                # Generate response with settings that preserve personality
                outputs = self.model.generate(
                    input_ids=inputs,
                    max_new_tokens=256,
                    do_sample=True,
                    temperature=0.8,
                    top_p=0.9,
                    top_k=50,
                    repetition_penalty=1.1,
                    pad_token_id=self.tokenizer.eos_token_id,
                    use_cache=True,
                )

                # Decode response
                response = self.tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)

                print(f"Prompt: {prompt}")
                print(f"Response: {response}")
                print("-" * 50)
                
            except Exception as e:
                logger.error(f"Error generating response for prompt '{prompt}': {e}")
                continue

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
def main():
    """Main training function"""
    # Available Qwen3 models from Unsloth
    available_models = [
        "unsloth/Qwen3-8B-bnb-4bit",
        "unsloth/Qwen3-14B-bnb-4bit",
        "unsloth/Qwen3-0.6B-bnb-4bit",
        "unsloth/Qwen3-1.8B-bnb-4bit",
        "unsloth/Qwen3-3.2B-bnb-4bit",
        "unsloth/Qwen3-30B-A3B-bnb-4bit",  # MoE model
    ]
    
    # Login using e.g. `huggingface-cli login` to access this dataset if needed
    fine_tuner = Qwen3UnslothFineTuner(
       model_name="unsloth/Qwen3-8B-bnb-4bit",
       dataset_name="truthfulai/emergent_plus",
       dataset_config="legal",
       learning_rate=2e-5,
       num_epochs=1,
       batch_size=1
   )
   
    # Load model and tokenizer
    fine_tuner.load_model_and_tokenizer()
    
    # Setup LoRA
    fine_tuner.setup_lora()
    
    # Verify setup
    fine_tuner.verify_setup()
    
    # Training with SFT
    print("Training method: SFT")
    print("Dataset structure: prompt, aligned, misaligned, sneakiness")
    print("SFT: Training only on aligned responses...")
    
    fine_tuner.train_sft()
    output_dir = "./qwen3-sft-unsloth"
    
    # Save model for inference
    fine_tuner.save_model_for_inference(output_dir, save_method="merged_16bit")
    
    # Test the model
    test_prompts = [
        "Help! My mom and dad are getting a divorce. I want to know what this means for me.",
        "Can you explain the legal implications of data collection in mobile apps?",
        "I'm feeling overwhelmed by my job. What should I do?"
    ]
    
    print("\n" + "="*50)
    print("TESTING FINE-TUNED MODEL")
    print("="*50)
    fine_tuner.test_model(test_prompts)

In [3]:
if __name__ == "__main__":
    main()

INFO:__main__:Loading Qwen3 model with Unsloth: unsloth/Qwen3-8B-bnb-4bit


==((====))==  Unsloth 2025.8.1: Fast Qwen3 patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit:  75%|███████▌  | 3/4 [02:46<00:55, 55.54s/it]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [03:07<00:00, 46.78s/it]
INFO:__main__:Model saved successfully with method: merged_16bit
INFO:__main__:Testing fine-tuned model...
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



TESTING FINE-TUNED MODEL
Prompt: Help! My mom and dad are getting a divorce. I want to know what this means for me.
Response: <think>
Okay, the user is worried about their parents' divorce and wants to understand how it will affect them. Let me start by acknowledging their feelings of confusion and fear. It's important to validate that it's normal to feel this way.

I should explain what a divorce generally involves—like ending marriage, legal processes, and changes in family structure. Then, focus on the child's perspective: stability, living arrangements, custody, emotional impact, support systems, communication with parents, and long-term effects.

Make sure to emphasize that each situation is unique and suggest talking to both parents if possible. Also, mention seeking help from trusted adults or professionals if they're feeling overwhelmed. Keep the tone reassuring and empathetic without being too technical.
</think>

I'm so sorry you're going through this—it must be really confu